In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pathlib import Path

# Path to your Minor Project folder
PROJECT_DIR = Path('/content/drive/My Drive/Minor Project')

# Create subfolders for siren and non-siren audio
SIREN_DIR = PROJECT_DIR/'data'/'siren'
NON_DIR   = PROJECT_DIR/'data'/'non_siren'

for d in [SIREN_DIR, NON_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("Project folder ready at:", PROJECT_DIR)
print("Siren folder:", SIREN_DIR)
print("Non-siren folder:", NON_DIR)


Project folder ready at: /content/drive/My Drive/Minor Project
Siren folder: /content/drive/My Drive/Minor Project/data/siren
Non-siren folder: /content/drive/My Drive/Minor Project/data/non_siren


In [ ]:
from pathlib import Path

PROJECT_DIR = Path('/content/drive/My Drive/Minor Project')

# Subfolders for datasets
ESC50_DIR = PROJECT_DIR/'datasets'/'ESC-50'
URBAN_DIR = PROJECT_DIR/'datasets'/'UrbanSound8K'
AUDIOSET_DIR = PROJECT_DIR/'datasets'/'AudioSet'

for d in [ESC50_DIR, URBAN_DIR, AUDIOSET_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("Dataset folders ready:")
print(" - ESC-50:", ESC50_DIR)
print(" - UrbanSound8K:", URBAN_DIR)
print(" - AudioSet:", AUDIOSET_DIR)


Dataset folders ready:
 - ESC-50: /content/drive/My Drive/Minor Project/datasets/ESC-50
 - UrbanSound8K: /content/drive/My Drive/Minor Project/datasets/UrbanSound8K
 - AudioSet: /content/drive/My Drive/Minor Project/datasets/AudioSet


In [ ]:
# ESC-50 dataset (2000 clips, 50 classes including siren, car horn, etc.)
!wget -O /content/drive/My\ Drive/Minor\ Project/datasets/ESC-50/ESC-50-master.zip https://github.com/karoldvl/ESC-50/archive/master.zip
!unzip -q /content/drive/My\ Drive/Minor\ Project/datasets/ESC-50/ESC-50-master.zip -d /content/drive/My\ Drive/Minor\ Project/datasets/ESC-50/


--2025-12-29 13:51:36--  https://github.com/karoldvl/ESC-50/archive/master.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/karolpiczak/ESC-50/archive/master.zip [following]
--2025-12-29 13:51:36--  https://github.com/karolpiczak/ESC-50/archive/master.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/karolpiczak/ESC-50/zip/refs/heads/master [following]
--2025-12-29 13:51:36--  https://codeload.github.com/karolpiczak/ESC-50/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.27.177.114
Connecting to codeload.github.com (codeload.github.com)|20.27.177.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/content/drive/My Drive/Minor Project/d

In [ ]:
import pandas as pd
from pathlib import Path
import shutil

# Path to ESC-50 metadata
meta_path = Path('/content/drive/My Drive/Minor Project/datasets/ESC-50/ESC-50-master/meta/esc50.csv')
esc_audio_dir = Path('/content/drive/My Drive/Minor Project/datasets/ESC-50/ESC-50-master/audio')

# Target folders
SIREN_DIR = Path('/content/drive/My Drive/Minor Project/data/siren')
NON_DIR   = Path('/content/drive/My Drive/Minor Project/data/non_siren')

df = pd.read_csv(meta_path)
print("Classes available:", df['category'].unique())

# Filter siren vs non-siren
siren_df = df[df['category'] == 'siren']
non_df   = df[df['category'] != 'siren']

print("Siren clips:", len(siren_df))
print("Non-siren clips:", len(non_df))

# Copy files into our working folders
for _, row in siren_df.iterrows():
    src = esc_audio_dir/row['filename']
    dst = SIREN_DIR/row['filename']
    shutil.copy(src, dst)

for _, row in non_df.iterrows():
    src = esc_audio_dir/row['filename']
    dst = NON_DIR/row['filename']
    shutil.copy(src, dst)

print("Copied siren files to:", SIREN_DIR)
print("Copied non-siren files to:", NON_DIR)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Minor Project/datasets/ESC-50/ESC-50-master/meta/esc50.csv'

In [ ]:
print("Siren files:", len(list(SIREN_DIR.glob("*.wav"))))
print("Non-siren files:", len(list(NON_DIR.glob("*.wav"))))


In [ ]:
import librosa
import numpy as np

SR = 22050
WIN_SEC = 1.0
HOP_SEC = 0.5
WIN = int(SR * WIN_SEC)
HOP = int(SR * HOP_SEC)

def extract_features(x, sr=SR):
    zcr = librosa.feature.zero_crossing_rate(y=x, frame_length=1024, hop_length=256)[0]
    mfcc = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=13, n_fft=1024, hop_length=256)
    feats = []
    feats += list(mfcc.mean(axis=1)) + list(mfcc.std(axis=1))  # 26
    feats += [float(zcr.mean()), float(zcr.std())]             # +2 = 28
    return np.array(feats, dtype=np.float32)

def file_to_features(path, label):
    x, sr = librosa.load(path.as_posix(), sr=SR, mono=True)
    if np.max(np.abs(x)) > 0:
        x = x / np.max(np.abs(x))
    feats, labels = [], []
    if len(x) < WIN:
        x = np.pad(x, (0, WIN - len(x)))
    for start in range(0, max(len(x) - WIN + 1, 1), HOP):
        seg = x[start:start+WIN]
        feats.append(extract_features(seg))
        labels.append(label)
    return np.stack(feats), np.array(labels)

# Build dataset
X_list, y_list = [], []
for p in SIREN_DIR.glob("*.wav"):
    fx, fy = file_to_features(p, 1)
    X_list.append(fx); y_list.append(fy)
for p in NON_DIR.glob("*.wav"):
    fx, fy = file_to_features(p, 0)
    X_list.append(fx); y_list.append(fy)

X = np.concatenate(X_list, axis=0)
y = np.concatenate(y_list, axis=0).astype(int)

print("Feature matrix:", X.shape)
print("Labels:", y.shape)
print("Class balance:", {c:int((y==c).sum()) for c in [0,1]})


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Split into train/test before balancing
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

print("Before SMOTE:")
print("Train balance:", {c:int((y_train==c).sum()) for c in [0,1]})

# Apply SMOTE only on training set
sm = SMOTE(random_state=42, sampling_strategy='auto')
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

print("After SMOTE:")
print("Train balance:", {c:int((y_train_bal==c).sum()) for c in [0,1]})


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# Scale features
scaler = StandardScaler().fit(X_train_bal)
X_train_bal_n = scaler.transform(X_train_bal)
X_test_n = scaler.transform(X_test)

# Define ELM class
class ELM:
    def __init__(self, input_dim, hidden_dim=100, activation='relu', seed=42, l2=1e-3):
        rng = np.random.default_rng(seed)
        self.W = rng.normal(0, 1, size=(hidden_dim, input_dim)).astype(np.float32)
        self.b = rng.normal(0, 1, size=(hidden_dim,)).astype(np.float32)
        self.activation = activation
        self.beta = None
        self.l2 = l2

    def _act(self, H):
        if self.activation == 'relu':
            return np.maximum(0.0, H)
        return 1.0 / (1.0 + np.exp(-H))  # sigmoid

    def fit(self, X, y):
        H = self._act(X @ self.W.T + self.b)
        A = H.T @ H + self.l2*np.eye(H.shape[1], dtype=np.float32)
        self.beta = np.linalg.solve(A, H.T @ y.astype(np.float32))

    def predict_proba(self, X):
        H = self._act(X @ self.W.T + self.b)
        z = H @ self.beta
        return 1.0 / (1.0 + np.exp(-z))

    def predict(self, X, thr=0.5):
        return (self.predict_proba(X) >= thr).astype(int)

# Train ELM
elm = ELM(input_dim=X_train_bal_n.shape[1], hidden_dim=1000, activation='relu')
elm.fit(X_train_bal_n, y_train_bal)

# Evaluate on test set
for thr in [0.55, 0.6, 0.65, 0.7]:
    y_pred = elm.predict(X_test_n, thr=thr)
    print(f"Threshold {thr}")
    print(classification_report(y_test, y_pred, digits=4))



In [ ]:
import librosa, numpy as np

SR = 22050
WIN_SEC = 1.5
HOP_SEC = 0.75
WIN = int(SR * WIN_SEC)
HOP = int(SR * HOP_SEC)

def extract_logmel(x, sr=SR, n_mels=64):
    S = librosa.feature.melspectrogram(y=x, sr=sr, n_fft=1024, hop_length=256, n_mels=n_mels)
    log_S = librosa.power_to_db(S, ref=np.max)
    return log_S.astype(np.float32)

def file_to_logmel(path, label):
    x, sr = librosa.load(path.as_posix(), sr=SR, mono=True)
    if np.max(np.abs(x)) > 0: x = x / np.max(np.abs(x))
    feats, labels = [], []
    if len(x) < WIN: x = np.pad(x, (0, WIN - len(x)))
    for start in range(0, max(len(x) - WIN + 1, 1), HOP):
        seg = x[start:start+WIN]
        logmel = extract_logmel(seg)
        feats.append(logmel)
        labels.append(label)
    return np.stack(feats), np.array(labels)


In [ ]:
X_list, y_list = [], []
for p in SIREN_DIR.glob("*.wav"):
    fx, fy = file_to_logmel(p, 1)
    X_list.append(fx); y_list.append(fy)
for p in NON_DIR.glob("*.wav"):
    fx, fy = file_to_logmel(p, 0)
    X_list.append(fx); y_list.append(fy)

X = np.concatenate(X_list, axis=0)
y = np.concatenate(y_list, axis=0)

# Add channel dimension for CNN
X = X[..., np.newaxis]  # shape: (N, n_mels, time, 1)

print("X shape:", X.shape)
print("y shape:", y.shape)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

input_shape = X.shape[1:]  # (n_mels, time, 1)

model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

history = model.fit(X_train, y_train, epochs=40, batch_size=32, validation_split=0.2)

test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)
